<h1><center>Softmax regression with batchnorm</h1></center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(threshold=np.nan, linewidth=140)

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import torch
from torch.autograd import Variable
from torch import Tensor

plotly.offline.init_notebook_mode(connected=True)

# 1.Forward prop for batchnorm

Forward propagation is straightforward:

$$
Z = W \cdot X; \\
Znorm = \frac{Z \circleddash \mu}{\sqrt{\sigma^{2}+\epsilon}}; \\
Zbn = Znorm \odot \gamma + \beta; \\
A   = softmax(Zbn). \\
$$

I used Signs $\circleddash, \odot$ to emphasize element-wise operations with possible broadcasting.

Slightly modified computational graph (to use simpler operations) for batchnorm layer, looks like this:

<center><h4>Computational graph for batchnorm layer</h4></center>
![computational graph for batchnorm layer](https://raw.githubusercontent.com/xtipacko/dl_class/master/5.NN%20Softmax%20and%20Batch%20Norm/batchnorm_forwardprop.png)


In [2]:
# toy dataset
K = 4
X = np.array([[1,1,0,0, 4,3,2,1, 7,6,5,4, 9,7,6,4],
              [8,6,4,3, 8,6,4,2, 6,5,4,3, 4,3,2,1]], dtype=np.float64)
Y = np.array([[0,0,0,0, 1,1,1,1, 2,2,2,2, 3,3,3,3]], dtype=np.float64)

deriv_tmp = dict()

In [3]:
#visualization
colors = ['rgba(252, 0, 0, .8)', 'rgba(0, 200, 0, .8)', 'rgba(0, 0, 200, .8)', 'rgba(200, 0, 200, .8)']
dots = []
for k in range(K):
    x = X[0,k*4:(k+1)*4]
    y = X[1,k*4:(k+1)*4]
    trace = go.Scatter(x = x, 
                       y = y,
                       name = f'class: {k}',
                       mode = 'markers',
                       marker = { 'size'  : 10,
                                  'color' : colors[k]} )

    dots.append(trace)

layout = dict(title = 'Dataset visualisation',
#               yaxis = dict(zeroline = False),
#               xaxis = dict(zeroline = False)
             )

fig = dict(data=dots, layout=layout)
    
plotly.offline.iplot(fig)

In [4]:
def one_hot_encoding(Yraw, K):
    H = np.arange(0, K).reshape(-1,1)
    Y = (H == Yraw).astype(np.int32)
    return Y

Y = one_hot_encoding(Y, K)

In [5]:
# initialization
def initialize():
    m = X.shape[1]
    l = [ X.shape[0], Y.shape[0] ]   

    beta = np.zeros((l[1],1), dtype=np.float64)
    gamma = np.ones((l[1],1), dtype=np.float64)
    W = (np.random.randn(l[1],l[0])*0.01).astype(np.float64)

    Zmean = np.zeros((Y.shape[0], 1), dtype=np.float64)
    Zstd  = np.zeros((Y.shape[0], 1), dtype=np.float64)
    return W,gamma,beta,Zmean,Zstd

In [6]:
def batchnorm(Z, gamma, beta):
    m = Z.shape[1]
    mean = np.mean(Z,axis=1,keepdims=True)
    Z0 = Z - mean
    var = np.sum(np.square(Z0), axis=1, keepdims=True) / m 
    std = np.sqrt(var+epsilon)
    
    Znorm = Z0 / std
    Zbn = Znorm * gamma + beta
    
    return Zbn, mean, Z0, var, std, Znorm


def softmax(Z):
    #D = np.max(Z, axis=0) #try to remove it if there will be a problem
    exponents = np.exp(Z)# - D)
    expsum = np.sum(exponents, axis=0, keepdims=True)
    return exponents / expsum


def cross_entropy(Y, H):
#     Y = Y.astype(np.bool)
#     Yhj = H[Y]
#     Losses = -np.log(Yhj[np.nonzero(Yhj)])
#     return np.sum(Losses) / H.shape[1]
    return np.mean(-np.sum(Y*np.log(H), axis=0, keepdims=True))

In [7]:
# forward
def forward(X,Y, W, gamma, beta):
    Z = W @ X 
    Zbn, Zmean, Z0, Zvar, Zstd, Znorm = batchnorm(Z, gamma, beta)
    A = softmax(Zbn)
    cost = cross_entropy(Y,A)
    cache = {'Z'    : Z, 
             'Znorm': Znorm,
             'Zbn'  : Zbn, 
             'Zmean': Zmean, 
             'Z0'   : Z0, 
             'Zvar' : Zvar, 
             'Zstd' : Zstd,
             'A'    : A,
             'cost' : cost}
    return cache    

In [8]:
# test mode
def test_batchnorm(Z, gamma, beta, Zmean, Zstd):   
    Znorm = (Z - Zmean) / Zstd    
    Zbn = Znorm * gamma + beta    
    return Zbn


def test_forward(X, W, gamma, beta, Zmean, Zstd):
    Z = W @ X 
    Zbn = test_batchnorm(Z, gamma, beta, Zmean, Zstd)
    A = softmax(Zbn)
    return A


def predict(X, W, gamma, beta, Zmean, Zstd):
    A = test_forward(X,W, gamma, beta, Zmean, Zstd)
    return np.argmax(A, axis=0)   

# 2. Derivative of batchnorm layer

After long calculations, we have these formulas.



$$~~~\begin{align*}\partial \beta_{n}  & = \sum_{i=0}^{m}Zbn_{n,i} \\
~~~\partial \gamma_{n} & = \sum_{i=0}^{m}\left(\partial Zbn \odot Znorm\right)_{n,i}\\
~~~\partial Znorm & = \partial Zbn \odot \gamma \\
\end{align*}$$

**Let:**

$~~~\mu = \frac{1}{m}\sum_{i=0}^{m}Z_{n,i};$

$~~~Z_{\varnothing} = Z\circleddash \mu;$

$~~~var = \frac{1}{m}\sum_{i=0}^{m}\left(Z_{\varnothing}^{2}\right)_{n,i};$

$~~~std = \sqrt{var + \epsilon};$


**Then:**

$$\begin{align*}
~~~\partial Z_{\varnothing} & = \left(\partial Znorm ~ \odot \frac{1}{std}\right)  + \left( \sum_{i=0}^{m} \left( \partial Znorm \odot Z_{\varnothing} \right)_{n,i}  \odot \frac{-1}{var + \epsilon} \odot \frac{1}{2 \cdot std} \odot \frac{\boldsymbol{J}}{m} \odot \left(2 \cdot Z_{\varnothing} \right) \right) \\
~~~\partial Z & = \partial Z_{\varnothing} \circleddash\frac{1}{m} \cdot \sum_{i=0}^{m}\partial Z_{\varnothing\left(n,i\right)}
\end{align*}$$ 

**Where:**

$~~~Z,~Z_{\varnothing},~\partial Z,~\partial Znorm  \in \mathbb{R}^{n,m},$

$~~~\mu,~var,~std  \in \mathbb{R}^{n,1},$

$~~~\gamma, \beta \in \mathbb{R}^{n,1},$

$~~~\boldsymbol{J} \in \begin{bmatrix}
        1 &      1  &  \dots  &       1 \\
        1 &      1  &  \dots  &       1 \\
   \vdots & \vdots  &  \ddots &  \vdots \\
        1 &      1  &  \dots  &       1 \\
\end{bmatrix}^{(n,m)}$

I used signs $\odot, \circleddash$ to emphasize element-wise operation with possible broadcasting.



In [9]:
def backward(X,Y, gamma, beta, cache):
    A     = cache['A']
    Z     = cache['Z']
    Znorm = cache['Znorm']
    Zstd  = cache['Zstd']
    Zvar  = cache['Zvar']
    Zmean = cache['Zmean']
    Z0    = cache['Z0']
    J     = np.ones(Z.shape)
    m     = X.shape[1]    
    
    dZbn = A - Y       # CORRECTLY  AND MORE STABLE dZbn = A - 1(Y == 1)
    dZbn /= m          # DIDN'T KNOW THIS
    dbeta = np.sum(dZbn, axis=1, keepdims=True)
    dgamma = np.sum(dZbn * Znorm, axis=1, keepdims=True)
    dZnorm = dZbn * gamma
    dZ0 = (dZnorm/Zstd) - Z0*(np.sum(dZnorm*Z0, axis=1, keepdims=True) / (m*(Zvar+epsilon)*Zstd))

    #TODO decompose and verify each component
#     dZ0 = (dZnorm/Zstd) + (np.sum(dZnorm*Z0, axis=1, keepdims=True)
#                            *(-np.reciprocal(Zvar+epsilon))
#                            *np.reciprocal(2*Zstd)
#                            *(J/m)
#                            *(2*Z0))
    dZ = dZ0 - np.sum(dZ0, axis=1, keepdims=True) / m

#     dZrstd = np.sum(dZnorm*Z0, axis=1, keepdims=True)
#     dZstd = dZrstd*(-np.reciprocal(Zvar+epsilon))
#     dZvar = dZstd*np.reciprocal(2*Zstd)    
#     dZ0squred = dZvar*(J/m)    
#     dZ0_std_branch = dZ0squred*(2*Z0)
#     dZ0_Znorm_branch = dZnorm/Zstd
#     dZ0 = dZ0_std_branch + dZ0_Znorm_branch
    
#     dZmean = -1 * np.sum(dZ0, axis=1, keepdims=True)
#     dZ_mean_branch = dZmean * (J/m)
#     dZ_Z0_branch = dZ0
#     dZ = dZ_mean_branch + dZ_Z0_branch 
    
    dW = dZ @ X.T
    
    return dW, dgamma, dbeta, dZbn, dZnorm, dZ0, dZ

In [10]:
def unpack_weights(theta_v, W, gamma, beta):
    pW = (theta_v[:W.size]).reshape(W.shape)
    pg = (theta_v[W.size:W.size+gamma.size]).reshape(gamma.shape)
    pb = (theta_v[W.size+gamma.size:]).reshape(beta.shape)
    return pW, pg, pb

def pack_weights(W, gamma, beta):   
    theta = np.concatenate((W.reshape(-1), gamma.reshape(-1), beta.reshape(-1)))
    return theta

def numerical_grad(X,Y, W, gamma, beta):
    thetasize = W.size + gamma.size + beta.size
    theta_plus = np.zeros(thetasize, dtype=np.float64)
    dtheta = np.zeros(thetasize, dtype=np.float64)

    
    for i in range(thetasize):
        theta_plus[i] = 1e-7 # 1e-7 = epsilon
        
        pW, pg, pb = unpack_weights(theta_plus, W, gamma, beta)
        leftcost  = forward(X,Y, W - pW, gamma - pg, beta - pb)['cost']
        rightcost = forward(X,Y, W + pW, gamma + pg, beta + pb)['cost']
        dtheta[i] = (rightcost - leftcost) / 2e-7  #2e-7 = 2*epsilon
        
        theta_plus[i] = 0
        
    dW, dgamma, dbeta = unpack_weights(dtheta, W, gamma, beta)
    return dW, dgamma, dbeta
    

In [11]:
def backhook(varname):
    def hook(var):
        global deriv_tmp
        deriv_tmp[varname] = var
    return hook
    
def torch_grad(X, Y, W, gamma, beta):
    tX = torch.from_numpy(X).type(new_type=torch.DoubleTensor)
    tY = torch.from_numpy(Y).type(new_type=torch.DoubleTensor)
    tW = torch.from_numpy(W).type(new_type=torch.DoubleTensor)
    tgamma = torch.from_numpy(gamma).type(new_type=torch.DoubleTensor)
    tbeta = torch.from_numpy(beta).type(new_type=torch.DoubleTensor)
    
    vX = Variable(tX, requires_grad=False)
    vY = Variable(tY, requires_grad=False)
    
    vW = Variable(tW,  requires_grad=True)
    vgamma = Variable(tgamma,  requires_grad=True)
    vbeta = Variable(tbeta,  requires_grad=True)
    
    Z = vW @ vX
    Z.register_hook(backhook('dZ'))
    Zmean = torch.mean(Z, 1, keepdim=True)
    Z0 = Z - Zmean
    Z0.register_hook(backhook('dZ0'))
    
    Zvar =  torch.mean((Z0**2), 1, keepdim=True)
    Zstd =  torch.sqrt(Zvar + epsilon)# does this work?
    
    Znorm = Z0 / Zstd
    Znorm.register_hook(backhook('dZnorm'))
    
    Zbn = Znorm*vgamma + vbeta    
    Zbn.register_hook(backhook('dZbn'))
    
    #softmax
    D, _ = torch.max(Zbn, 0, keepdim=True)
    exps = torch.exp(Zbn - D)
    expsums = torch.sum(exps,0, keepdim=True)
    A = exps / expsums
    #np.mean(-np.sum(Y*np.log(H), axis=0, keepdims=True))
    cost = torch.mean(-torch.sum(vY*torch.log(A), 0, keepdim=True))
    
#     vW.grad.data.zero_()
#     vgamma.grad.data.zero_()
#     vbeta.grad.data.zero_()
    
    cost.backward()
    
    dW = vW.grad.data.numpy()
    dgamma = vgamma.grad.data.numpy()
    dbeta = vbeta.grad.data.numpy()
    dZbn = deriv_tmp['dZbn'].data.numpy()
    dZnorm = deriv_tmp['dZnorm'].data.numpy()
    dZ0 = deriv_tmp['dZ0'].data.numpy()
    dZ = deriv_tmp['dZ'].data.numpy()
    
    return cost.data[0], dW, dgamma, dbeta, dZbn, dZnorm,dZ0, dZ, Zmean.data.numpy(), Zstd.data.numpy()

In [12]:
def check_eror(param1, param2):
#     return  np.linalg.norm(dtheta - ndtheta) / (np.linalg.norm(dtheta) + np.linalg.norm(ndtheta))
    param1_flat = param1.reshape(-1)
    param2_flat = param2.reshape(-1)
    errors = np.abs(param1_flat - param2_flat) / np.maximum(np.abs(param1_flat), np.abs(param2_flat))
    return np.mean(errors)

In [13]:
def train(X, Y, W, gamma, beta, Zmean, Zstd, numeric=False, analytic=True, autograd=False, learning_rate=0.01, iterations=1000, report=50):
    assert numeric or analytic or autograd, 'at least one method should be chosen'
    for i in range(iterations):        
        if numeric:
            ndW, ndgamma, ndbeta = numerical_grad(X,Y, W, gamma, beta)

        if autograd:
            tcost, tdW, tdgamma, tdbeta, tdZbn, tdZnorm, tdZ0, tdZ, tZmean, tZstd = torch_grad(X, Y, W, gamma, beta)
            
            
        if analytic:
            cache = forward(X,Y,W,gamma,beta)
            dW, dgamma, dbeta, dZbn, dZnorm, dZ0, dZ = backward(X,Y, gamma, beta, cache)
            
            Zmean = 0.7*Zmean + 0.3*cache['Zmean']
            Zstd  = 0.7*Zstd  + 0.3*cache['Zstd']
        elif numeric:
            dW, dgamma, dbeta = ndW, ndgamma, ndbeta
            cache = forward(X,Y,W,gamma,beta)
        elif autograd:
            dW, dgamma, dbeta = tdW, tdgamma, tdbeta
            Zmean = 0.7*Zmean + 0.3*tZmean
            Zstd = 0.7*Zstd + 0.3*tZstd
            cache = {'cost':tcost}
            

        if not i%report:
            print(f'[{i:>06}] Cost: {cache["cost"]:.7f}')
            if numeric and analytic and autograd: 
                dtheta = pack_weights(dW, dgamma, dbeta)
                ndtheta = pack_weights(ndW, ndgamma, ndbeta)
                tdtheta = pack_weights(tdW, tdgamma, tdbeta)
                
                anal_num__err = check_eror(dtheta, ndtheta)
                auto_anal_err = check_eror(dtheta, tdtheta)
                num_auto__err = check_eror(ndtheta, tdtheta)
                
                anal_num__dW_err = check_eror(dW, ndW)
                auto_anal_dW_err = check_eror(dW, tdW)
                num_auto__dW_err = check_eror(ndW, tdW)
                
                anal_num__dg_err = check_eror(dgamma, ndgamma)
                auto_anal_dg_err = check_eror(dgamma, tdgamma)
                num_auto__dg_err = check_eror(ndgamma, tdgamma)
                
                anal_num__db_err = check_eror(dbeta, ndbeta)
                auto_anal_db_err = check_eror(dbeta, tdbeta)
                num_auto__db_err = check_eror(ndbeta, tdbeta)
                

                auto_anal_dZbn_err = check_eror(dZbn, tdZbn)
                auto_anal_dZnorm_err = check_eror(dZnorm, tdZnorm)
                auto_anal_dZ0_err = check_eror(dZ0, tdZ0)
                auto_anal_dZ_err = check_eror(dZ, tdZ)
                auto_anal_dW_err = check_eror(dW, tdW)
    
                                
                print(f'[{i:>06}] Analytical-Numerical error: {anal_num__err:.8f} | dW {anal_num__dW_err:.8f}| dgamma {anal_num__dg_err:.8f} | dbeta {anal_num__db_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical  error: {auto_anal_err:.8f} | dW {auto_anal_dW_err:.8f}| dgamma {auto_anal_dg_err:.8f} | dbeta {auto_anal_db_err:.8f}')
                print(f'[{i:>06}] Numerical-Autograd   error: {num_auto__err:.8f} | dW {num_auto__dW_err:.8f}| dgamma {num_auto__dg_err:.8f} | dbeta {num_auto__db_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical dZbn error {auto_anal_dZbn_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical dZnorm error {auto_anal_dZnorm_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical dZ0 error {auto_anal_dZ0_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical dZ error {auto_anal_dZ_err:.8f}')
                print(f'[{i:>06}] Autograd-Analytical dW error {auto_anal_dW_err:.8f}')

        W -= learning_rate*dW
        gamma -= learning_rate*dgamma
        beta -= learning_rate*dbeta

    return W, gamma, beta, Zmean, Zstd

In [14]:
epsilon = 1e-8
W,gamma,beta,Zmean,Zstd = initialize()
W, gamma, beta, Zmean, Zstd = train(X,Y,W,gamma,beta,Zmean,Zstd, numeric=False, analytic=True, autograd = False,
                                    learning_rate=1, iterations=30000, report=3000)

[000000] Cost: 1.8488977
[003000] Cost: 0.0170106
[006000] Cost: 0.0081240
[009000] Cost: 0.0053181
[012000] Cost: 0.0039517
[015000] Cost: 0.0031448
[018000] Cost: 0.0026124
[021000] Cost: 0.0022349
[024000] Cost: 0.0019533
[027000] Cost: 0.0017351


In [15]:
x1, x2 = np.meshgrid(np.arange(0, 10, 0.1),
                     np.arange(0, 10, 0.1))
dp = np.row_stack((x1.ravel(),x2.ravel()))

prediction = predict(dp, W, gamma, beta, Zmean, Zstd)

prediction = prediction.reshape(100,100)


cntr =  {'z': prediction,
        'colorscale':'Jet',
        'type': u'contour',
        'dx': 0.1,
        'x0': 0,
        'dy': 0.1,
        'y0': 0 }

data = go.Data([go.Contour(cntr)])

data = data + dots

layout = dict(title = 'Decision surfaces visualisation')

fig = dict(data=data, layout=layout)
    
plotly.offline.iplot(fig)